In [1]:
import requests
import json
import sqlite3
from sqlite3 import Error
import pandas as pd

In [ ]:
headers = {
    'x-rapidapi-key': "38bab05df5msh928211553580d9cp181186jsne583ad5bd5cd",
    'x-rapidapi-host': "free-nba.p.rapidapi.com"
    }
page = 1
results = []
while True:


    url = "https://free-nba.p.rapidapi.com/stats"

    querystring = {"page": page,"per_page":"100"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    data = response.json()
    if len(data['data']) <= 0:
        break
    results += data['data']
    print(response.json())
    break

    page += 1

    print(response.json())



In [ ]:
with open("data.json", "w") as data_file:
    json.dump(results, data_file, indent=4)
    data_file.close()

In [ ]:
print(data["data"][0]['fg3_pct'])

In [6]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        c = conn.cursor()
        print(sqlite3.version)
        c.execute('''CREATE TABLE players (PLAYER_NAME text, TEAM_ID text, PLAYER_ID text, SEASON int)''')
        players = pd.read_csv('players.csv')
        players.to_sql('players', conn, if_exists='append', index=False)
        c.execute('''SELECT * FROM players''').fetchall()

    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


if __name__ == '__main__':
    create_connection(r"nba_test.db")

2.6.0
table players already exists


In [9]:
import sqlite3

conn = sqlite3.connect('nba_test')
c = conn.cursor()

In [10]:
c.execute('''CREATE TABLE players (PLAYER_NAME text, TEAM_ID text, PLAYER_ID text, SEASON int)''')

In [12]:
players = pd.read_csv('players.csv')
players.to_sql('players', conn, if_exists='append', index=False)


In [13]:
c.execute('''SELECT * FROM players LIMIT 5''').fetchall()

[("Royce O'Neale", '1610612762', '1626220', 2019),
 ('Bojan Bogdanovic', '1610612762', '202711', 2019),
 ('Rudy Gobert', '1610612762', '203497', 2019),
 ('Donovan Mitchell', '1610612762', '1628378', 2019),
 ('Mike Conley', '1610612762', '201144', 2019)]

In [14]:
teams = pd.read_csv('teams.csv')
teams.to_sql('teams', conn, if_exists='append', index=False)
games = pd.read_csv('games.csv')
games.to_sql('games', conn, if_exists='append', index=False)
games_details = pd.read_csv('games_details.csv')
games_details.to_sql('games_details', conn, if_exists='append', index=False)